# Simple Calculator 

In [1]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QGridLayout, QPushButton, QLineEdit

class Calculator(QWidget):
    def __init__(self):
        super().__init__()

        # Set the window title
        self.setWindowTitle("Simple Calculator")
        
        # Set the layout of the calculator
        self.layout = QVBoxLayout()
        
        # Create the display line edit
        self.display = QLineEdit()
        self.display.setReadOnly(True)
        self.display.setFixedHeight(35)
        self.layout.addWidget(self.display)
        
        # Create the grid for buttons
        self.gridLayout = QGridLayout()

        # Button labels and positions
        buttons = [
            ('7', 1, 0), ('8', 1, 1), ('9', 1, 2), ('/', 1, 3),
            ('4', 2, 0), ('5', 2, 1), ('6', 2, 2), ('*', 2, 3),
            ('1', 3, 0), ('2', 3, 1), ('3', 3, 2), ('-', 3, 3),
            ('0', 4, 0), ('C', 4, 1), ('=', 4, 2), ('+', 4, 3),
        ]
        
        # Add buttons to the grid layout
        for text, row, col in buttons:
            button = QPushButton(text)
            button.clicked.connect(self.on_button_click)
            self.gridLayout.addWidget(button, row, col)
        
        self.layout.addLayout(self.gridLayout)
        self.setLayout(self.layout)

        self.current_input = ""

    def on_button_click(self):
        # Get the text of the clicked button
        button = self.sender()
        text = button.text()
        
        if text == 'C':
            # Clear the display and input
            self.display.setText("")
            self.current_input = ""
        elif text == '=':
            # Evaluate the input expression
            try:
                result = str(eval(self.current_input))
                self.display.setText(result)
                self.current_input = result  # Update the current input with the result
            except Exception as e:
                self.display.setText("Error")
        else:
            # Append the button text to the current input
            self.current_input += text
            self.display.setText(self.current_input)

if __name__ == "__main__":
    app = QApplication(sys.argv)

    calculator = Calculator()
    calculator.show()

    sys.exit(app.exec_())


SystemExit: 0

C:\Users\mazen\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Integrating Open CV with PyQt5

In [1]:
import sys
import cv2
import threading # to isolate processes of GUI and CV 
from PyQt5.QtWidgets import QApplication, QLabel, QVBoxLayout, QWidget, QPushButton
from PyQt5.QtCore import Qt, QTimer
from PyQt5.QtGui import QImage, QPixmap


class CameraApp(QWidget):
    def __init__(self):
        super().__init__()
        
        # Set window title
        self.setWindowTitle("Webcam Viewer")
        
        # Create a vertical layout
        self.layout = QVBoxLayout()
        
        # Create a label to display the camera feed
        self.label = QLabel("Press 'Start' to access the webcam")
        self.label.setAlignment(Qt.AlignCenter)
        self.layout.addWidget(self.label)
        
        # Create buttons for start/stop webcam
        self.start_button = QPushButton("Start Webcam")
        self.stop_button = QPushButton("Stop Webcam")
        self.start_button.clicked.connect(self.start_camera)
        self.stop_button.clicked.connect(self.stop_camera)
        
        # Add buttons to the layout
        self.layout.addWidget(self.start_button)
        self.layout.addWidget(self.stop_button)
        
        # Set the layout for the window
        self.setLayout(self.layout)
        
        # Initialize camera capture variables
        self.cap = None
        self.is_running = False

    def start_camera(self):
        """Start the webcam and display frames."""
        if not self.is_running:
            self.is_running = True
            self.cap = cv2.VideoCapture(0)
            self.thread = threading.Thread(target=self.update_frame, daemon=True)
            self.thread.start()
    
    def stop_camera(self):
        """Stop the webcam."""
        self.is_running = False
        if self.cap is not None:
            self.cap.release()
            self.cap = None
        self.label.setText("Webcam stopped")

    def update_frame(self):
        """Continuously capture frames and update the GUI."""
        while self.is_running:
            ret, frame = self.cap.read()
            if ret:
                # Convert frame from BGR (OpenCV) to RGB (for PyQt)
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                
                # Convert to QImage
                h, w, ch = rgb_frame.shape
                bytes_per_line = ch * w
                qt_image = QImage(rgb_frame.data, w, h, bytes_per_line, QImage.Format_RGB888)
                
                # Set the pixmap to the label
                self.label.setPixmap(QPixmap.fromImage(qt_image))

if __name__ == "__main__":
    app = QApplication(sys.argv)
    
    # Create the GUI
    window = CameraApp()
    window.show()
    
    # Run the app
    sys.exit(app.exec_())


SystemExit: 0

C:\Users\mazen\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
import sys
import pygame
import threading
import queue
from PyQt5.QtWidgets import QApplication, QLabel, QVBoxLayout, QWidget

# Initialize the Queue for sharing data between threads
data_queue = queue.Queue()

# Function to run Pygame in a separate thread
def run_pygame():
    pygame.init()
    screen = pygame.display.set_mode((640, 480))
    pygame.display.set_caption('Pygame Window')

    clock = pygame.time.Clock()

    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

            # Check for keydown events to trigger updates in PyQt5
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_q:
                    data_queue.put("Key Q pressed!")
                elif event.key == pygame.K_w:
                    data_queue.put("Key W pressed!")
                elif event.key == pygame.K_a:
                    data_queue.put("Key A pressed!")
                elif event.key == pygame.K_s:
                    data_queue.put("Key S pressed!")
                elif event.key == pygame.K_ESCAPE:
                    running = False

        # Fill the screen with a color (just for visual)
        screen.fill((0, 128, 255))

        # Update the display
        pygame.display.flip()

        # Maintain frame rate
        clock.tick(30)

    pygame.quit()

# PyQt5 GUI class
class AppWindow(QWidget):
    def __init__(self):
        super().__init__()

        # Set window title
        self.setWindowTitle("PyQt5 GUI")

        # Create a vertical layout
        self.layout = QVBoxLayout()

        # Create a label to display data received from Pygame
        self.label = QLabel("Press keys in Pygame and see the results here!")
        self.layout.addWidget(self.label)

        # Set layout
        self.setLayout(self.layout)

        # Start the timer to update the label with data from Pygame
        self.timer = self.startTimer(100)  # 100ms interval

    def timerEvent(self, event):
        # Check if there is any data in the queue
        if not data_queue.empty():
            data = data_queue.get()
            self.label.setText(data)

# Function to start both PyQt5 and Pygame
def run_gui():
    app = QApplication(sys.argv)
    window = AppWindow()
    window.show()

    # Start the Pygame thread
    pygame_thread = threading.Thread(target=run_pygame)
    pygame_thread.start()

    # Run the PyQt5 event loop
    sys.exit(app.exec_())

if __name__ == "__main__":
    run_gui()


SystemExit: 0